In [1]:
import torch
import torch.nn as nn

In [2]:
BATCH_SIZE=4
SEQUENCE_LENGTH=5
INPUT_SIZE=2
HIDDEN_SIZE=3
NUM_LAYERS=1
BIDERECTIONAL=True
D = 2 if BIDERECTIONAL == True else 1

In [3]:
rnn = nn.RNN(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, bidirectional=BIDERECTIONAL)

In [4]:
# ---------------------------- #
# weights forward
# ---------------------------- #

# input to hidden weights and biases
w_ih_l0 = rnn.weight_ih_l0
b_ih_l0 = rnn.bias_ih_l0

# hidden to hidden weights and biases
w_hh_l0 = rnn.weight_hh_l0
b_hh_l0 = rnn.bias_hh_l0

# ---------------------------- #
# weights reverse
# ---------------------------- #

# input to hidden weights and biases
w_r_ih_l0 = rnn.weight_ih_l0_reverse
b_r_ih_l0 = rnn.bias_ih_l0_reverse

# hidden to hidden weights and biases
w_r_hh_l0 = rnn.weight_hh_l0_reverse
b_r_hh_l0 = rnn.bias_hh_l0_reverse

In [5]:
sequence = torch.randn(SEQUENCE_LENGTH, BATCH_SIZE, INPUT_SIZE)
h_0 = torch.zeros(D * NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE)

In [6]:
sequence.size()

torch.Size([5, 4, 2])

In [7]:
h_0.size()

torch.Size([2, 4, 3])

In [8]:
with torch.inference_mode():
    output, h_n = rnn(sequence, h_0)

In [9]:
output.shape

torch.Size([5, 4, 6])

In [10]:
h_n.shape

torch.Size([2, 4, 3])

In [11]:
def manual_rnn():
    hidden = h_0.clone()
    output = torch.zeros(SEQUENCE_LENGTH, BATCH_SIZE, D * HIDDEN_SIZE)
    with torch.inference_mode():
        for idx in range(SEQUENCE_LENGTH):
            hidden[0] = torch.tanh(sequence[idx] @ w_ih_l0.T + b_ih_l0 + hidden[0] @ w_hh_l0.T + b_hh_l0)
            hidden[1] = torch.tanh(sequence[SEQUENCE_LENGTH - 1 -idx] @ w_r_ih_l0.T + b_r_ih_l0 + hidden[1] @ w_r_hh_l0.T + b_r_hh_l0)
            output[idx, :, :HIDDEN_SIZE] = hidden[0]
            output[SEQUENCE_LENGTH - 1 - idx, :, HIDDEN_SIZE:] = hidden[1]
    return output, hidden